In [1]:
from keras.applications import MobileNet

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

Using TensorFlow backend.


0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [2]:
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 2

FC_Head = lw(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'images/train'
validation_data_dir = 'images/test'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [5]:
train_generator.class_indices

{'amit': 0, 'preeti': 1}

In [6]:
validation_generator.class_indices

{'amit': 0, 'preeti': 1}

In [7]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("FaceRecognition_mobileNet.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 800
nb_validation_samples = 200

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save("faceClassifier.h5")

Epoch 1/5
50/50 [==============================] - 145s 3s/step - loss: 1.9256 - accuracy: 0.8131 - val_loss: 1.2542 - val_accuracy: 0.5056

Epoch 00001: val_loss improved from inf to 1.25425, saving model to FaceRecognition_mobileNet.h5
Epoch 2/5
50/50 [==============================] - 149s 3s/step - loss: 0.2976 - accuracy: 0.9125 - val_loss: 0.1836 - val_accuracy: 0.8839

Epoch 00002: val_loss improved from 1.25425 to 0.18360, saving model to FaceRecognition_mobileNet.h5
Epoch 3/5
50/50 [==============================] - 130s 3s/step - loss: 0.1988 - accuracy: 0.9544 - val_loss: 0.2607 - val_accuracy: 0.8512

Epoch 00003: val_loss did not improve from 0.18360
Epoch 4/5
50/50 [==============================] - 140s 3s/step - loss: 0.1284 - accuracy: 0.9556 - val_loss: 0.0224 - val_accuracy: 0.9639

Epoch 00004: val_loss improved from 0.18360 to 0.02236, saving model to FaceRecognition_mobileNet.h5
Epoch 5/5
50/50 [==============================] - 135s 3s/step - loss: 0.2964 - accur

In [8]:
from keras.models import load_model

classifier = load_model('FaceRecognition_mobileNet.h5')

In [9]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = {"[0]": "Amit Chaudhary" , 
                      "[1]": "Preeti Chaudhary"
                     }

monkey_breeds_dict_n = {"amit": "Amit Chaudhary ", 
                      "preeti": "Preeti Chaudhary"}
        
def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("images/test/")
    
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()


Class - Amit Chaudhary 
Class - Amit Chaudhary 
Class - Preeti Chaudhary
Class - Preeti Chaudhary
Class - Amit Chaudhary 
Class - Amit Chaudhary 
Class - Preeti Chaudhary
Class - Preeti Chaudhary
Class - Amit Chaudhary 
Class - Preeti Chaudhary


In [10]:
from keras.models import load_model

In [11]:
model = load_model("FaceClassifier.h5")


In [12]:
import numpy as np
import os
import IPython.display as ipd
import cv2
from keras.preprocessing import image
import PIL

In [13]:
face_classifier = cv2.CascadeClassifier('C:/Users/amitc/Desktop/MLOps Training Data/Face Recognition/haarcascade_frontalface_default.xml')

In [14]:
def face_detector(img, size=0.5):
    faces = face_classifier.detectMultiScale(img,1.3,5)
    if faces is ():
        return img, []
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0, 255, 255), 2)
        roi = img[y:y+h,x:x+w]
        roi = cv2.resize(roi, (224, 224))
    return img, roi

In [15]:
cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    image, face = face_detector(frame)
    try:
        face = np.array(face)
        face = np.expand_dims(face,axis = 0)
        if(face.shape== (1,0)):
            cv2.putText(image, "Not Detected", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255),2)
            cv2.imshow('Face Recognition',image)
            print("3")
        else:
            results = model.predict(face)
            if(results[0][0] == 0.0):
                cv2.putText(image, "Amit Chaudhary", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255),2)
                cv2.imshow('Face Recognition',image)
                print("2")
            elif(results[0][0] == 0.1):
                cv2.putText(image, "Preeti Chaudhary", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255),2)
                cv2.imshow('Face Recognition',image)
                print("1")
            elif(results[0][0] == 0.2):
                cv2.putText(image, "Priya Chaudhary", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255),2)
                cv2.imshow('Face Recognition',image)
                print("0")
                
    except:
        cv2.imshow('Face Recognition',image)
        cv2.putText(image, "No Face Found", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255),2)
        cv2.putText(image, "Looked", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255),2)
        print("4")
    if cv2.waitKey(1) == 13:
        break
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'


In [16]:
cap.release()